### Import

In [1]:
import os

In [2]:
from selenium import webdriver
from langdetect import detect

In [3]:
import requests

In [4]:
from bs4 import BeautifulSoup

In [5]:
import pandas as pd

In [9]:
from tqdm import tqdm

In [10]:
from googletrans import Translator

In [11]:
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate

ModuleNotFoundError: No module named 'google.cloud'

In [354]:
credentials = service_account.Credentials.from_service_account_file('cred.json')
translate_client = translate.Client(credentials=credentials)

### Scraping - final test

In [355]:
# PROXY = "23.23.23.23:3128" # IP:PORT or HOST:PORT
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=http://%s' % PROXY)


In [356]:
# driver = webdriver.Chrome("driver/chromedriver", chrome_options=chrome_options)
driver = webdriver.Chrome("driver/chromedriver")

In [357]:
DATA_PATH = 'reviews-data/'
positive_path = 'positive/'
negative_path = 'negative/'

In [358]:
LINK_ROOT = 'https://hotline.ua/yp/2476/reviews/'
positive_link = '?dt=3&asf=1&p='
negative_link = '?dt=3&asf=2&p='

In [359]:
# test
os.path.join(LINK_ROOT, positive_link)

'https://hotline.ua/yp/2476/reviews/?dt=3&asf=1&p='

In [360]:
LINK = os.path.join(LINK_ROOT, positive_link) + str(2)
LINK

'https://hotline.ua/yp/2476/reviews/?dt=3&asf=1&p=2'

In [361]:
driver.get(LINK)

In [362]:
content = driver.page_source
soup = BeautifulSoup(content)

In [363]:
divs = soup.find_all("div", class_="viewbox")

In [364]:
for div in divs:
    p_tag = div.find_all("p", class_="break-word")
    if p_tag != []:
        text = p_tag[0].text
        if detect(text) == 'uk':
            print(text)
            print("-"*50)
    else:
        None

Замовлення (903383816) було зроблене минулої весни, але нам сподобалося ставлення – нам зателефонували і сказали, що достатньої кількості ламп немає, тож запропонували почекати тиждень, зробили знижку 15% та безкоштовну доставку.
Також, перед новим роком ми замовляли навушники (351521507), час доставки було визначено у дуже широкому інтервалі, і коли кур'єр приїхав – нас не було на місці. Він погодився приїхати ще раз після десятої вечора з власної ініціятиви, за що йому велике спасибі.
Щоб уникнути таких випадків, вашим кур'єрам треба телефонувати за годину чи дві до приїзду (ви ж однаково плануєте маршрут), а потім іще раз – за 10 хвилин. Або можна надсилати повідомлення. Зекономите час та нерви і кур'єрам, і клієнтам.
Нюанси, які треба виправити:
- Розетка виписала чек на ім'я власника платіжної картки, а не на ім'я отримувача, яке було вказане під час замовлення, через що довелося "вгадувати ім'я" під час доставки.
- Оплата через сайт працює радше як замаскований переказ з картки н

### Final scraping

In [365]:
def collect_data(link, file_path, start_number):
    # preparation
    driver.get(link)
    # get page html content
    content = driver.page_source
    soup = BeautifulSoup(content)
    # get div with reviews
    divs = soup.find_all("div", class_="viewbox")
    review_number = 0
    # get each reviews
    for div in divs:
        p_tag = div.find_all("p", class_="break-word")
        if p_tag != []:
            text = p_tag[0].text
            # detecr language [uk]
            if detect(text) == 'uk':
                file_name = os.path.join(file_path, 'review-%d.txt'%start_number)
                with open(file_name, "w") as f:
                    f.write(text)
                review_number += 1
                start_number += 1
            else:
                try:
                    translator = Translator()
                    translated_text = translate_client.translate(text, source_language='ru', target_language='uk')
                    file_name = os.path.join(file_path, 'translated-review-%d.txt'%start_number)
                    with open(file_name, "w") as f:
                        f.write(translated_text['translatedText'])
                    review_number += 1
                    start_number += 1
                except:
                    print('some error :(')
                
                
    return start_number, review_number

In [366]:
# positive
# Rozetka
"""
start_index = 0
for i in range(285):
    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, positive_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nstart_index = 0\nfor i in range(285):\n    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, positive_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [367]:
# negative (from 182)
# Rozetka
"""
start_index = 1907
for i in range(398, 630):
    
    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, negative_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nstart_index = 1907\nfor i in range(398, 630):\n    \n    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, negative_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [368]:
# possible links:
# Цитрус: https://hotline.ua/yp/2248/reviews/
# MOYO: https://hotline.ua/yp/11054/reviews/
# STYLUS: https://hotline.ua/yp/287/reviews/
# ФОКСТРОТ: https://hotline.ua/yp/27343/reviews/

In [369]:
# positive
# Цитрус
"""
LINK_ROOT = 'https://hotline.ua/yp/2248/reviews/'

start_index = 926
for i in range(210, 330):
    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, positive_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/2248/reviews/'\n\nstart_index = 926\nfor i in range(210, 330):\n    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, positive_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [370]:
# negative
# cutrys

"""
LINK_ROOT = 'https://hotline.ua/yp/2248/reviews/'
start_index = 2048
for i in range(170):
    
    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, negative_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/2248/reviews/'\nstart_index = 2048\nfor i in range(170):\n    \n    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, negative_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [371]:
# MOYO: https://hotline.ua/yp/11054/reviews/

# positive
"""
LINK_ROOT = 'https://hotline.ua/yp/11054/reviews/'

start_index = 685
for i in range(285):
    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, positive_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/11054/reviews/'\n\nstart_index = 685\nfor i in range(285):\n    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, positive_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [372]:
# negative

"""
LINK_ROOT = 'https://hotline.ua/yp/11054/reviews/'
start_index = 2150
for i in range(140):
    
    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, negative_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/11054/reviews/'\nstart_index = 2150\nfor i in range(140):\n    \n    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, negative_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [373]:
# STYLUS: https://hotline.ua/yp/287/reviews/

# positive
"""
LINK_ROOT = 'https://hotline.ua/yp/287/reviews/'

start_index = 715
for i in range(285):
    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, positive_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/287/reviews/'\n\nstart_index = 715\nfor i in range(285):\n    LINK = os.path.join(LINK_ROOT, positive_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, positive_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"

In [1]:
# negative

"""
LINK_ROOT = 'https://hotline.ua/yp/287/reviews/'

start_index = 2260
for i in range(190):
    
    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)
    index, review_number = collect_data(
        link=LINK, 
        file_path=os.path.join(DATA_PATH, negative_path), 
        start_number=start_index
    )
    start_index = index
    print('Iteration: ', i, 'writen: ', review_number)
"""

"\nLINK_ROOT = 'https://hotline.ua/yp/287/reviews/'\n\nstart_index = 2260\nfor i in range(190):\n    \n    LINK = os.path.join(LINK_ROOT, negative_link)+str(i)\n    index, review_number = collect_data(\n        link=LINK, \n        file_path=os.path.join(DATA_PATH, negative_path), \n        start_number=start_index\n    )\n    start_index = index\n    print('Iteration: ', i, 'writen: ', review_number)\n"